In [5]:
import pandas as pd
import requests
import urllib.parse
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

font = {'family' : 'normal',
        'size'   : 11}

matplotlib.rc('font', **font)

base_endpoint = 'https://opendata.nhsbsa.net/api/3/action/'
package_list_method = 'package_list'     # List of data-sets in the portal
package_show_method = 'package_show?id=' # List all resources of a data-set
action_method = 'datastore_search_sql?'  # SQL action method
resource_name = "EPD_202205"

## Count of ADHD meds prescribed

base_endpoint = 'https://opendata.nhsbsa.net/api/3/action/'
package_list_method = 'package_list'     # List of data-sets in the portal
package_show_method = 'package_show?id=' # List all resources of a data-set
action_method = 'datastore_search_sql?'  # SQL action method
resource_name = "EPD_202205"
adhd_single_month_query = "SELECT ICB_NAME,sum(TOTAL_QUANTITY) AS count_adhd " \
                 f"FROM `{resource_name}` " \
                 f"WHERE bnf_chemical_substance LIKE '0404000__' " \
                 f"GROUP BY ICB_NAME " \
                 f"ORDER BY icb_name "

adhd_single_month_api_call = f"{base_endpoint}" \
                    f"{action_method}" \
                    "resource_id=" \
                    f"{resource_name}" \
                    "&" \
                    "sql=" \
                    f"{urllib.parse.quote(adhd_single_month_query)}"

adhd_single_month_response = requests.get(adhd_single_month_api_call).json()

adhd_df = pd.json_normalize(adhd_single_month_response['result']['result']['records'])

print(adhd_df)
adhd_df.drop(adhd_df[adhd_df['ICB_NAME'] == 'UNIDENTIFIED'].index, inplace = True)

## Count of all meds prescribed

non_adhd_single_month_query = "SELECT ICB_NAME,sum(TOTAL_QUANTITY) AS count " \
                 f"FROM `{resource_name}` " \
                 f"GROUP BY ICB_NAME " \
                 f"ORDER BY icb_name "

non_adhd_single_month_api_call = f"{base_endpoint}" \
                    f"{action_method}" \
                    "resource_id=" \
                    f"{resource_name}" \
                    "&" \
                    "sql=" \
                    f"{urllib.parse.quote(non_adhd_single_month_query)}"

non_adhd_single_month_response = requests.get(non_adhd_single_month_api_call).json()

non_adhd_df = pd.json_normalize(non_adhd_single_month_response['result']['result']['records'])

## Derive % total

percent_total = 100* adhd_df['count_adhd'] / non_adhd_df['count']
adhd_df['non_adhd'] = non_adhd_df['count']

adhd_df['percent_total'] = percent_total
adhd_df_sorted = adhd_df.sort_values(by=['percent_total'],ascending=False)
adhd_df_sorted['percent_total_index'] = range(len(adhd_df_sorted))

## Adding population data

populations_pre = pd.read_csv("populations.csv")
populations = populations_pre[['CODE','NUMBER_OF_PATIENTS']]

single_month_query = "SELECT DISTINCT icb_name,practice_code " \
                 f"FROM `{resource_name}` "

single_month_api_call = f"{base_endpoint}" \
                    f"{action_method}" \
                    "resource_id=" \
                    f"{resource_name}" \
                    "&" \
                    "sql=" \
                    f"{urllib.parse.quote(single_month_query)}"

single_month_response = requests.get(single_month_api_call).json()

df = pd.json_normalize(single_month_response['result']['result']['records'])

df1 = df.merge(populations,how='left',left_on='practice_code', right_on='CODE')
df1 = df1.groupby('icb_name').sum()

adhd_df_pop = adhd_df_sorted.merge(df1,how='left',left_on='ICB_NAME',right_on='icb_name')
adhd_df_pop = adhd_df_pop.rename(str.lower, axis='columns')

### Calculate ADHD meds prescribed per person

adhd_df_pop['adhd_per_pop'] = adhd_df_pop['count_adhd']/adhd_df_pop['number_of_patients']
adhd_df_pop_sorted = adhd_df_pop.sort_values(by=['adhd_per_pop'],ascending=False)
adhd_df_pop_sorted['adhd_per_pop_index'] = range(len(adhd_df_pop_sorted))

## Adding GDHI data

gdhi = pd.read_csv('gdhi.csv')
gdhi = gdhi[['icb_name','GDHI/head']]
adhd_df_gdhi = adhd_df_pop_sorted.merge(gdhi,how='left',left_on='icb_name',right_on='icb_name')
adhd_df_gdhi_sorted = adhd_df_gdhi.sort_values(by=['GDHI/head'],ascending=False)
adhd_df_gdhi_sorted['GDHI/head_index'] = range(len(adhd_df_gdhi_sorted))

## Adding unemployment data

unemployment = pd.read_csv('unemployment.csv')
adhd_df_unemployment = adhd_df_gdhi_sorted.merge(unemployment,how='left',left_on='icb_name',right_on='icb_name')
adhd_df_unemployment_sorted = adhd_df_unemployment.sort_values(by=['unemployment'],ascending=False)
adhd_df_unemployment_sorted['unemployment_index'] = range(len(adhd_df_unemployment_sorted))

## Adding life expectancy data

life = pd.read_csv('life_expectancy.csv')
life['avg_life_expectancy'] = (life['f_life_expectancy'] + life['m_life_expectancy'])/2
adhd_df_life = adhd_df_unemployment_sorted.merge(life,how='left',left_on='icb_name',right_on='icb_name')
adhd_df_life_sorted = adhd_df_life.sort_values(by=['avg_life_expectancy'],ascending=False)
adhd_df_life_sorted['life_expectancy_index'] = range(len(adhd_df_life_sorted))

## Adding lifestyle data

lifestyle = pd.read_csv('lifestyle.csv')
adhd_df_lifestyle = adhd_df_life_sorted.merge(lifestyle,how='left',left_on='icb_name',right_on='icb_name')
variables = adhd_df_lifestyle[['percent_total','adhd_per_pop','GDHI/head','unemployment','avg_life_expectancy','anxiety','happiness','life_satisfaction','worthwhile']]
plt.figure(figsize=(12,6))
sns.heatmap(variables.corr()[:,:5],vmin=-1, vmax=1, annot=True,cmap='vlag')
plt.savefig('Figures/correlation')
plt.show()

adhd_df_sorted = adhd_df_lifestyle.sort_values(by=['icb_name'],ascending=True)
adhd_df_sorted.to_csv('adhd.csv')

plt.figure(figsize=(8.6,6))
plt.subplot(2,1,1)
print(variables.corr())
sns.heatmap(variables.corr()[0:2],vmin=-1, vmax=1, annot=True,cmap='vlag')
plt.yticks(np.arange(2)+0.5,('percent_total','adhd_per_pop'),
           rotation=0, fontsize="10", va="center")
plt.savefig("Figures/correlation_pair")
plt.show()

                                    ICB_NAME  count_adhd
0   NHS BATH AND NORTH EAST SOMERSET, SWINDO    106603.0
1   NHS BEDFORDSHIRE, LUTON AND MILTON KEYNE    139135.0
2   NHS BIRMINGHAM AND SOLIHULL INTEGRATED C    137050.0
3    NHS BLACK COUNTRY INTEGRATED CARE BOARD    142436.0
4   NHS BRISTOL, NORTH SOMERSET AND SOUTH GL    120272.0
5   NHS BUCKINGHAMSHIRE, OXFORDSHIRE AND BER    256158.0
6   NHS CAMBRIDGESHIRE AND PETERBOROUGH INTE    111767.0
7   NHS CHESHIRE AND MERSEYSIDE INTEGRATED C    480573.0
8   NHS CORNWALL AND THE ISLES OF SCILLY INT     73913.0
9   NHS COVENTRY AND WARWICKSHIRE INTEGRATED    105494.0
10  NHS DERBY AND DERBYSHIRE INTEGRATED CARE    155829.0
11           NHS DEVON INTEGRATED CARE BOARD    156229.0
12          NHS DORSET INTEGRATED CARE BOARD     99987.0
13         NHS FRIMLEY INTEGRATED CARE BOARD     97538.0
14  NHS GLOUCESTERSHIRE INTEGRATED CARE BOAR     57905.0
15  NHS GREATER MANCHESTER INTEGRATED CARE B    353345.0
16  NHS HAMPSHIRE AND ISLE OF W

KeyboardInterrupt: 